In [372]:
import numpy as np
import pandas as pd
import yfinance as yf
import random
import warnings
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")


In [373]:
def fill_missing_values(column, method='ffill'):

    if column.isnull().any():
        if method == 'zero':
            return column.fillna(0)
        elif method == 'interpolate':
            return column.interpolate(method='linear').fillna(method='ffill').fillna(method='bfill')
        else:
            return column.fillna(method='ffill').fillna(method='bfill')

    return column


In [374]:
def calculate_rsi(df, period=14):
    delta = df['Close'].diff()

    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=period, min_periods=1).mean()
    avg_loss = loss.rolling(window=period, min_periods=1).mean()

    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))

    return df

In [375]:
def calculate_ema(df, column='Close', period=14):
    alpha = 2 / (period + 1)
    ema = [None] * len(df)
    prices = df[column].values
    ema[period - 1] = sum(prices[:period]) / period
    for i in range(period, len(prices)):
        ema[i] = alpha * prices[i] + (1 - alpha) * ema[i - 1]
    return pd.Series(ema, index=df.index)

def calculate_atr(df, period=14):
    high_low = df['High'] - df['Low']
    high_close = np.abs(df['High'] - df['Close'].shift(1))
    low_close = np.abs(df['Low'] - df['Close'].shift(1))
    tr = np.maximum.reduce([high_low, high_close, low_close])
    tr = pd.Series(tr, index=df.index)
    atr = tr.rolling(window=period).mean()
    df['ATR'] = atr
    df['ATR'] = fill_missing_values(df['ATR'])
    return df

In [421]:
# Helpful for detecting Trend Reversals

def calculate_psar(df, af_start=0.02, af_increment=0.02, af_max=0.2):
    psar = [None] * len(df)
    psar_direction = 1  # 1 for uptrend, -1 for downtrend
    af = af_start
    ep = df['High'][0] if psar_direction == 1 else df['Low'][0]

    psar[0] = df['Low'][0] if psar_direction == 1 else df['High'][0]

    for i in range(1, len(df)):
        if psar_direction == 1:
            psar[i] = psar[i - 1] + af * (ep - psar[i - 1])

            if df['Low'][i] < psar[i]:
                psar_direction = -1
                psar[i] = ep
                af = af_start
                ep = df['Low'][i]
            else:
                if df['High'][i] > ep:
                    ep = df['High'][i]
                    af = min(af + af_increment, af_max)
                psar[i] = min(psar[i], df['Low'][i - 1], df['Low'][i - 2] if i > 1 else psar[i])
        else:
            psar[i] = psar[i - 1] + af * (ep - psar[i - 1])

            if df['High'][i] > psar[i]:
                psar_direction = 1
                psar[i] = ep
                af = af_start
                ep = df['High'][i]
            else:
                if df['Low'][i] < ep:
                    ep = df['Low'][i]
                    af = min(af + af_increment, af_max)
                psar[i] = max(psar[i], df['High'][i - 1], df['High'][i - 2] if i > 1 else psar[i])

    df['PSAR'] = psar
    return df


In [377]:
def calculate_macd(df, column='Close', short_period=12, long_period=26, signal_period=9):
    df = df.copy()

    short_ema = calculate_ema(df, column=column, period=short_period)
    short_ema = fill_missing_values(short_ema, method='interpolate')

    long_ema = calculate_ema(df, column=column, period=long_period)
    long_ema = fill_missing_values(long_ema, method='interpolate')

    macd_line = short_ema - long_ema
    df['MACD_Line'] = macd_line
    df['MACD_Line'] = fill_missing_values(df['MACD_Line'], method='interpolate')

    signal_line = calculate_ema(df, column='MACD_Line', period=signal_period)
    df['Signal_Line'] = signal_line
    df['Signal_Line'] = fill_missing_values(df['Signal_Line'], method='interpolate')

    df['MACD_Histogram'] = df['MACD_Line'] - df['Signal_Line']

    df['MACD_Histogram'] = fill_missing_values(df['MACD_Histogram'], method='interpolate')

    return df

In [378]:
def reduce_noise(data):
    process_variance=1e-5
    measurement_variance=1
    posteri_estimate = 0.0   #  X(t+1|t) or Y(t+1|t)
    posteri_error_estimate = 1.0  # P(t+1|t)

    smoothed_prices = []
    for price in data['Close']:
      priori_estimate = posteri_estimate  #𝑋𝑡+1|𝑡 = 𝜙𝑋𝑡|𝑡 + 𝑐𝑡 or 𝑌𝑡+1|𝑡 = 𝐻𝑋𝑡 + 𝑑𝑡
      priori_error_estimate = posteri_error_estimate + process_variance #𝑃𝑡+1|𝑡 = 𝜙𝑃𝑡|𝑡 𝜙 (𝑇) + 𝑄

      kalman_gain=priori_error_estimate+measurement_variance           #𝐾𝑡+1 = 𝑃𝑡+1|𝑡 𝐻 (𝑇) [𝐻𝑃𝑡+1|𝑡 𝐻 (𝑇) + 𝑅]^-1
      posteri_estimate= priori_estimate+kalman_gain * (price - priori_estimate)   #𝑋𝑡+1 = 𝑋𝑡+1|𝑡 + 𝐾𝑡+1 (𝑌𝑡+1 − 𝑌𝑡+1|𝑡)
      posteri_error_estimate = (1 - kalman_gain) * priori_error_estimate  #𝑃𝑡+1|𝑡+1 = [𝐼 − 𝐾𝑡+1 𝐻]𝑃𝑡+1|𝑡
      smoothed_prices.append(posteri_estimate)

    data['smoothed_close'] = smoothed_prices
    return data

In [440]:
def process_data(data):
    data = reduce_noise(data)
    data = calculate_psar(data)

    data['EMA9'] = calculate_ema(data, column='Close', period=9)
    data['EMA9'] = fill_missing_values(data['EMA9'])
    data['EMA20'] = calculate_ema(data, column='Close', period=20)
    data['EMA20'] = fill_missing_values(data['EMA20'])

    data = calculate_rsi(data)
    data['RSI'] = data['RSI'].fillna(40)


    data['P_EMA_RSI'] = calculate_ema(data, column='RSI', period=5)
    data['P_EMA_RSI'] = fill_missing_values(data['P_EMA_RSI'])

    data = calculate_macd(data)
    data = calculate_atr(data)

    return data


In [441]:
def prepare_data(symbol):
    data = yf.download(symbol, start='2020-01-01', end='2022-12-31', interval='5d')
    data.columns = data.columns.droplevel(1)
    return data


In [442]:
btc_data = prepare_data('BTC-USD')
eth_data = prepare_data('ETH-USD')

btc_data=process_data(btc_data)
eth_data=process_data(eth_data)
data = pd.DataFrame({'ETH': eth_data['Close'], 'BTC': btc_data['Close']}).dropna()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [443]:
null_counts = btc_data.isnull().sum()
print(null_counts)

Price
Adj Close         0
Close             0
High              0
Low               0
Open              0
Volume            0
smoothed_close    0
PSAR              0
EMA9              0
EMA20             0
RSI               0
P_EMA_RSI         0
MACD_Line         0
Signal_Line       0
MACD_Histogram    0
ATR               0
dtype: int64


In [444]:
print(btc_data['MACD_Line'])

Date
2020-01-01     852.278255
2020-01-06     852.278255
2020-01-11     852.278255
2020-01-16     852.278255
2020-01-21     852.278255
                 ...     
2022-12-06   -2136.464110
2022-12-11   -2050.899343
2022-12-16   -1996.921973
2022-12-21   -1918.309760
2022-12-26   -1826.699663
Name: MACD_Line, Length: 219, dtype: float64


In [445]:
# Dynamic Time Warping (DTW) for Cross-correlation

def fastdtw(x, y, radius=1):
    n, m = len(x), len(y)
    cost = np.full((n + 1, m + 1), np.inf)
    cost[0, 0] = 0

    for i in range(1, n + 1):
        for j in range(max(1, i - radius), min(m, i + radius) + 1):
            cost[i, j] = abs(x[i - 1] - y[j - 1]) + min(
                cost[i - 1, j],  # insertion
                cost[i, j - 1],  # deletion
                cost[i - 1, j - 1]  # match
            )

    path = []
    i, j = n, m
    while i > 0 and j > 0:
        path.append((i - 1, j - 1))
        if cost[i - 1, j] == min(cost[i - 1, j], cost[i, j - 1], cost[i - 1, j - 1]):
            i -= 1
        elif cost[i, j - 1] == min(cost[i - 1, j], cost[i, j - 1], cost[i - 1, j - 1]):
            j -= 1
        else:
            i -= 1
            j -= 1
    path.reverse()
    return cost[n, m], path

In [446]:
window_size = 30
dtw_distances = []
for i in range(len(data) - window_size):
    eth_window = data['ETH'].iloc[i:i + window_size].values
    btc_window = data['BTC'].iloc[i:i + window_size].values
    distance, _ = fastdtw(eth_window, btc_window)
    dtw_distances.append(distance)


dtw_threshold = np.percentile(dtw_distances, 25)
data = data.iloc[window_size:]
data['DTW_Distance'] = dtw_distances

In [447]:
print(len(data))

189


In [387]:
print(btc_data[['PSAR', 'Close']][103:113])

Price               PSAR         Close
Date                                  
2021-05-30  58769.072384  35678.128906
2021-06-04  56749.205656  36894.406250
2021-06-09  54890.928266  37345.121094
2021-06-14  52649.421963  40218.476562
2021-06-19  50632.066290  35615.871094
2021-06-24  48816.446184  34662.437500
2021-06-29  46844.698424  35867.777344
2021-07-04  45109.560394  35287.781250
2021-07-09  43582.638928  33798.011719
2021-07-14  42005.712798  32822.347656


In [448]:
def strategy_self(data, thresholds):
    """
    '0' : for No Action Taken
    '2' : for Long Entry
    '-2' : for Long Exit
    '1' : for Short Entry
    '-1' : for Short Exit
    """
    data['signals'] = 0
    sample_number = 0
    total_samples = len(data)
    position = "NONE"  # No position initially

    action_taken = 0
    threshold_low, threshold_high = thresholds
    while sample_number < total_samples:
        action_taken = 0
        cur_row = data.iloc[sample_number]
        prev_row = data.iloc[sample_number - 1] if sample_number > 0 else cur_row

        if cur_row['P_EMA_RSI'] > threshold_high:
            market_scenario = "Uptrend"
        elif cur_row['P_EMA_RSI'] < threshold_low:
            market_scenario = "Downtrend"
        else:
            market_scenario = "Fluctuating"


        if market_scenario == "Fluctuating":
            if cur_row['RSI'] > 55 and prev_row['RSI'] > cur_row['RSI']:  # Weakening Momentum - Bearish Reversal
                action_taken = 1
                if position == "NONE":
                    data.at[cur_row.name, 'signals'] = 1
                    position = "SHORT"
                elif position == 'LONG':
                    data.at[cur_row.name, 'signals'] = -2
                    position = "NONE"
            elif cur_row['RSI'] < 45 and prev_row['RSI'] < cur_row['RSI']:    # Weakening Momentum - Bullish Reversal
                action_taken = 1
                if position == "NONE":
                    data.at[cur_row.name, 'signals'] = 2
                    position = "LONG"
                elif position == 'SHORT':
                    data.at[cur_row.name, 'signals'] = -1
                    position = "NONE"


        elif market_scenario == "Downtrend":
            if cur_row['PSAR'] <= cur_row['Close'] and prev_row['PSAR'] >= cur_row['PSAR']:
                action_taken = 1
                if position == "NONE":
                    data.at[cur_row.name, 'signals'] = 2
                    position = "LONG"
                elif position == 'SHORT':
                    data.at[cur_row.name, 'signals'] = -1
                    position = "NONE"

            # Hammer Condition
            elif (cur_row['Low'] < cur_row['Open']) and (cur_row['Low'] < cur_row['Close']) and \
              (cur_row['Close'] - cur_row['Low'] > 2 * abs(cur_row['Open'] - cur_row['Close'])) and \
              (cur_row['High'] - cur_row['Close'] < 0.3 * (cur_row['Close'] - cur_row['Low'])):
                action_taken = 1
                if position == "NONE":
                    data.at[cur_row.name, 'signals'] = 2
                    position = "LONG"
                elif position == 'SHORT':
                    data.at[cur_row.name, 'signals'] = -1
                    position = "NONE"

            else:
                action_taken = 1
                if position == "NONE":
                    data.at[cur_row.name, 'signals'] = 1
                    position = "SHORT"
                elif position == 'LONG':
                    data.at[cur_row.name, 'signals'] = -2
                    position = "NONE"


        elif market_scenario == "Uptrend":
            if cur_row['EMA9'] > cur_row['EMA20']:
                action_taken = 1
                if position == "NONE":
                    data.at[cur_row.name, 'signals'] = 2
                    position = "LONG"
                elif position == 'SHORT':
                    data.at[cur_row.name, 'signals'] = -1
                    position = "NONE"
            else:
                action_taken = 1
                if position == "NONE":
                    data.at[cur_row.name, 'signals'] = 1
                    position = "SHORT"
                elif position == 'LONG':
                    data.at[cur_row.name, 'signals'] = -2
                    position = "NONE"


        # Additional cross strategy if no action taken
        if not action_taken:
            res = strategy_cross(sample_number)
            if res == 1:
                if position == "NONE":
                    data.at[cur_row.name, 'signals'] = 2
                    position = "LONG"
                elif position == 'SHORT':
                    data.at[cur_row.name, 'signals'] = -1
                    position = "NONE"
            elif res == -1:
                if position == "NONE":
                    data.at[cur_row.name, 'signals'] = 1
                    position = "SHORT"
                elif position == 'LONG':
                    data.at[cur_row.name, 'signals'] = -2
                    position = "NONE"

        sample_number += 1

    if position == "LONG":
        data.at[data.index[-1], 'signals'] = -2  # Exit Long position
    elif position == "SHORT":
        data.at[data.index[-1], 'signals'] = -1  # Exit Short position

    return data


In [449]:
def strategy_cross(sample_no):
    if (sample_no + window_size >= len(eth_data)):
      return 0
    eth_row = eth_data.iloc[sample_no]
    prev_eth_row = eth_data.iloc[sample_no - 1] if sample_no > 0 else eth_row

    btc_row = btc_data.iloc[sample_no]
    prev_btc_row = btc_data.iloc[sample_no - 1] if sample_no > 0 else btc_row

    dtw_row = data.iloc[sample_no]
    prev_dtw_row = data.iloc[sample_no - 1] if sample_no > 0 else dtw_row


    if dtw_row['DTW_Distance'] < dtw_threshold:
        if (btc_row['ATR'] > prev_btc_row['ATR'] and eth_row['ATR'] > prev_eth_row['ATR']):
            if eth_row['MACD_Histogram'] > 0 and eth_row['Close'] > eth_row['EMA20']:
                return 1  # LONG
            elif eth_row['MACD_Histogram'] < 0 and eth_row['Close'] < eth_row['EMA20']:
                return -1  # SHORT

    return 0


In [450]:
def calculate_performance_metrics(df, initial_capital=10000):
    """
    Calculate comprehensive trading strategy performance metrics
    """


    trades = []
    current_trade = None

    portfolio_values = [initial_capital]
    current_capital = initial_capital

    peak_capital = initial_capital
    max_drawdown = 0
    max_adverse_excursion = 0

    for i in range(len(df)):
        if df['signals'].iloc[i] == 2:
            if current_trade is None:
                current_trade = {
                    'type': 'long',
                    'entry_price': df['Close'].iloc[i],
                    'entry_date': df.index[i],
                    'entry_signal_index': i
                }

        elif df['signals'].iloc[i] == 1:
            if current_trade is None:
                current_trade = {
                    'type': 'short',
                    'entry_price': df['Close'].iloc[i],
                    'entry_date': df.index[i],
                    'entry_signal_index': i
                }

        elif df['signals'].iloc[i] == -2 and current_trade and current_trade['type'] == 'long':
            exit_price = df['Close'].iloc[i]
            trade_return = (exit_price - current_trade['entry_price']) / current_trade['entry_price']

            # Received capital
            trade_capital = current_capital * (1 + trade_return)

            current_trade['exit_price'] = exit_price
            current_trade['exit_date'] = df.index[i]
            current_trade['return'] = trade_return
            trades.append(current_trade)

            # Update current capital and reset trade
            current_capital = trade_capital
            current_trade = None

        elif df['signals'].iloc[i] == -1 and current_trade and current_trade['type'] == 'short':
            exit_price = df['Close'].iloc[i]
            trade_return = (current_trade['entry_price'] - exit_price) / current_trade['entry_price']

            # Received capital
            trade_capital = current_capital * (1 + trade_return)

            current_trade['exit_price'] = exit_price
            current_trade['exit_date'] = df.index[i]
            current_trade['return'] = trade_return
            trades.append(current_trade)

            current_capital = trade_capital
            current_trade = None

        portfolio_values.append(current_capital)

        if current_capital > peak_capital:
            peak_capital = current_capital

        current_drawdown = (peak_capital - current_capital) / peak_capital
        max_drawdown = max(max_drawdown, current_drawdown)

    trades_df = pd.DataFrame(trades)

    metrics = {
        'Total Trades': len(trades),
        'Win Rate (%)': calculate_win_rate(trades_df),
        'Net Profit (%) (Compounded)': calculate_net_profit(initial_capital, current_capital),
        'Sharpe Ratio': calculate_sharpe_ratio(np.diff(portfolio_values)),
        'Sortino Ratio': calculate_sortino_ratio(np.diff(portfolio_values)),
        'Risk-Reward Ratio': calculate_risk_reward_ratio(trades_df),
        'Max Drawdown (%)': max_drawdown * 100,
        'Max Adverse Excursion (%)': calculate_max_adverse_excursion(trades_df),
        'Max Trade Duration (days)': calculate_max_trade_duration(trades_df),
        'Average Trade Duration (days)': calculate_avg_trade_duration(trades_df),
    }

    return metrics

def calculate_win_rate(trades_df):
    wins = trades_df[trades_df['return'] > 0].shape[0]
    return (wins / len(trades_df)) * 100 if len(trades_df) > 0 else 0

def calculate_net_profit(initial_capital, final_capital):
    return (final_capital - initial_capital) / initial_capital * 100

def calculate_sharpe_ratio(daily_returns):
    mean_return = np.mean(daily_returns)
    std_dev = np.std(daily_returns)
    return mean_return / std_dev if std_dev != 0 else 0

def calculate_sortino_ratio(daily_returns):
    downside_returns = [r for r in daily_returns if r < 0]
    downside_deviation = np.std(downside_returns)
    mean_return = np.mean(daily_returns)
    return mean_return / downside_deviation if downside_deviation != 0 else 0

def calculate_risk_reward_ratio(trades_df):


    avg_win = np.mean(trades_df[trades_df['return'] > 0]['return'])
    avg_loss = np.mean(trades_df[trades_df['return'] < 0]['return'])
    return avg_win / abs(avg_loss) if avg_loss != 0 else 0

def calculate_max_adverse_excursion(trades_df):

    return np.min(trades_df['return']) if len(trades_df) > 0 else 0

def calculate_max_trade_duration(trades_df):
    return (trades_df['exit_date'] - trades_df['entry_date']).max().days if len(trades_df) > 0 else 0

def calculate_avg_trade_duration(trades_df):
    return (trades_df['exit_date'] - trades_df['entry_date']).mean().days if len(trades_df) > 0 else 0


In [451]:
import numpy as np
import random

class Particle:
    def __init__(self, dim, position_bounds, velocity_bounds):
        lower = 45.0
        upper = 70.0
        self.position = np.array([lower, upper])
        self.velocity = np.array([random.uniform(velocity_bounds[0], velocity_bounds[1]) for _ in range(dim)])
        self.best_position = self.position.copy()
        self.best_fitness = float('-inf')

    def ensure_position_bounds(self, position_bounds):
        self.position[0] = np.clip(self.position[0], position_bounds[0], self.position[1] - 1)
        self.position[1] = np.clip(self.position[1], self.position[0] + 1, position_bounds[1])


def fitness_function(metrics):
    net_profit = (metrics['Net Profit (%) (Compounded)'])/100
    max_drawdown = (metrics['Max Drawdown (%)'])/100
    win_rate = (metrics['Win Rate (%)'])/100

    return net_profit - 0.5 * max_drawdown + 0.5 * win_rate

def evaluate_fitness(position, data):
    thresholds = position
    strategy_data = strategy_self(data.copy(), thresholds)
    metrics = calculate_performance_metrics(strategy_data)
    return fitness_function(metrics)


def particle_swarm_optimization(dim, data, n_particles=20, n_iterations=100, position_bounds=(30, 80), velocity_bounds=(-1, 1)):
    particles = [Particle(dim, position_bounds, velocity_bounds) for _ in range(n_particles)]
    global_best_position = particles[0].position.copy()
    global_best_fitness = float('-inf')
    inertia_weight = 0.7
    cognitive_coeff = 1.5
    social_coeff = 1.5

    for _ in range(n_iterations):
        for particle in particles:
            fitness = evaluate_fitness(particle.position, data)
            if fitness > particle.best_fitness:
                particle.best_fitness = fitness
                particle.best_position = particle.position.copy()
            if fitness > global_best_fitness:
                global_best_fitness = fitness
                global_best_position = particle.position.copy()

            cognitive_velocity = cognitive_coeff * random.random() * (particle.best_position - particle.position)
            social_velocity = social_coeff * random.random() * (global_best_position - particle.position)
            particle.velocity = inertia_weight * particle.velocity + cognitive_velocity + social_velocity
            particle.position += particle.velocity
            particle.ensure_position_bounds(position_bounds)

    return global_best_position, global_best_fitness


In [452]:
def plot_heikin_ashi_with_signals(eth_data, signals):
    heikin_ashi = eth_data.copy()
    heikin_ashi['HA-Close'] = (eth_data['Open'] + eth_data['High'] + eth_data['Low'] + eth_data['Close']) / 4
    heikin_ashi['HA-Open'] = (eth_data['Open'] + eth_data['Close']) / 2
    for i in range(1, len(heikin_ashi)):
        heikin_ashi['HA-Open'].iloc[i] = (heikin_ashi['HA-Open'].iloc[i - 1] + heikin_ashi['HA-Close'].iloc[i - 1]) / 2
    heikin_ashi['HA-High'] = heikin_ashi[['High', 'HA-Open', 'HA-Close']].max(axis=1)
    heikin_ashi['HA-Low'] = heikin_ashi[['Low', 'HA-Open', 'HA-Close']].min(axis=1)

    signal_dates = heikin_ashi.index[:len(signals)]

    long_entry_signals = [i for i, signal in enumerate(signals) if signal == 2]
    long_exit_signals = [i for i, signal in enumerate(signals) if signal == -2]
    short_signals = [i for i, signal in enumerate(signals) if signal == 1]
    short_exit_signals = [i for i, signal in enumerate(signals) if signal == -1]

    fig = go.Figure(data=[
        go.Candlestick(
            x=heikin_ashi.index,
            open=heikin_ashi['HA-Open'],
            high=heikin_ashi['HA-High'],
            low=heikin_ashi['HA-Low'],
            close=heikin_ashi['HA-Close'],
            increasing_line_color='green',
            decreasing_line_color='red'
        )
    ])

    fig.add_trace(go.Scatter(
        x=heikin_ashi.index[long_entry_signals],
        y=heikin_ashi['HA-Close'].iloc[long_entry_signals],
        mode='markers',
        marker=dict(color='darkblue', size=12, symbol='triangle-up'),
        name='Long Entry Signal'
    ))

    fig.add_trace(go.Scatter(
        x=heikin_ashi.index[long_exit_signals],
        y=heikin_ashi['HA-Close'].iloc[long_exit_signals],
        mode='markers',
        marker=dict(color='darkred', size=12, symbol='triangle-down'),
        name='Long Exit Signal'
    ))

    fig.add_trace(go.Scatter(
        x=heikin_ashi.index[short_signals],
        y=heikin_ashi['HA-Close'].iloc[short_signals],
        mode='markers',
        marker=dict(color='darkorange', size=12, symbol='triangle-left'),
        name='Short Signal'
    ))

    fig.add_trace(go.Scatter(
        x=heikin_ashi.index[short_exit_signals],
        y=heikin_ashi['HA-Close'].iloc[short_exit_signals],
        mode='markers',
        marker=dict(color='purple', size=12, symbol='triangle-right'),
        name='Short Exit Signal'
    ))

    fig.update_layout(
        title='Ethereum (ETH-USD) Heikin-Ashi Candlestick Chart with Trade Signals',
        xaxis_title='Date',
        yaxis_title='Price (USD)',
        xaxis_rangeslider_visible=False
    )

    fig.show()


In [453]:
best_thresholds, best_fitness = particle_swarm_optimization(dim=2, n_particles=30, n_iterations=100, data=btc_data)
signals=strategy_self(btc_data,best_thresholds)

In [454]:
plot_heikin_ashi_with_signals(btc_data,signals['signals'])